In [185]:
#Vérifier qu'on est bien sur Python 3.5
import sys
sys.version_info

sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)

In [186]:
import numpy as np
from scipy.stats import truncnorm
import math

In [187]:
import data_extraction_script_module2

In [188]:
tp = data_extraction_script_module2.Temperatures()
tp.get_temperatures('C:/Users/roman/Desktop/ENSAE 3A/HMM et Monte Carlo/Donnees temperature/')

In [189]:
tp.df.head()

hour,00:00,03:00,06:00,09:00,12:00,15:00,18:00,21:00,date
0,3.012821,2.725641,2.317949,2.625641,4.176923,4.641026,3.333333,2.753846,2011-01-01
1,2.553846,2.415385,2.143590,2.566667,4.584615,4.387179,2.282051,1.587179,2011-01-02
2,0.653846,-0.351282,-0.892308,-0.023077,2.430769,2.689744,0.700000,-0.164103,2011-01-03
3,-0.248718,-0.705128,-0.666667,-0.492308,1.961538,2.661538,1.223077,0.671795,2011-01-04
4,0.476923,0.402564,0.435897,1.933333,5.351282,6.128205,5.682051,6.064103,2011-01-05


In [190]:
elec = data_extraction_script_module2.Electricity()
elec.get_electricity_data("C:/Users/roman/Desktop/ENSAE 3A/HMM et Monte Carlo/Donnees electricite/")

C:\Users\roman\Desktop\ENSAE 3A\HMM et Monte Carlo\data_extraction_script_module2.py:144: UserWarning: WARNING : There is more than 365 days or less than 365 days in dataframe
  warnings.warn("WARNING : There is more than 365 days or less than 365 days in dataframe")


Historique_consommation_INST_2012/Historique_consommation_INST_2012.xls
Number of rows 366


In [191]:
elec.df.head()

,Date,Type de données,00:30,01:00,01:30,02:00,02:30,03:00,03:30,04:00,...,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,24:00,Day_type
0,2011-01-01,Définitives,70671.0,68887.0,69045.0,68830.0,68143.0,65950.0,64343.0,62676.0,...,70926.0,70532.0,69433.0,68257.0,67221.0,68287.0,70497.0,69320.0,69062.0,6
1,2011-01-02,Définitives,67537.0,65099.0,65152.0,64785.0,64478.0,62594.0,61319.0,60084.0,...,75851.0,74972.0,73480.0,71938.0,70788.0,71380.0,73374.0,72138.0,71897.0,7
2,2011-01-03,Définitives,70483.0,68325.0,68628.0,68694.0,68715.0,67143.0,66038.0,65140.0,...,86672.0,84611.0,82317.0,80219.0,78181.0,78450.0,80072.0,78675.0,78509.0,0
3,2011-01-04,Définitives,76712.0,74313.0,74466.0,74199.0,74196.0,72428.0,71160.0,70052.0,...,88879.0,86587.0,84053.0,81787.0,79995.0,80119.0,81749.0,80214.0,79953.0,1
4,2011-01-05,Définitives,78121.0,75826.0,75627.0,75486.0,75425.0,73532.0,72180.0,71197.0,...,85901.0,83329.0,80561.0,78396.0,76036.0,75946.0,77336.0,75885.0,75385.0,1


On prend 15:00 pour tester (au moins pas de problèmes de changement d'heure et autre)

In [192]:
#Create date series only to keep track of the dates
date_ts = elec.date_ts()
#Create Day Type series
daytype_ts = elec.daytype_ts()
#Create temperature time series at the chosen hour
tp_ts = tp.temperature_ts('15:00')
#Create electricity time series at the chosen hour
elec_ts = elec.elec_ts('15:00')

In [79]:
np.random.seed(1234)
#Take arbitrary parameters
u_heat = np.random.normal(size=1, loc = 14, scale = 1)[0] #loc = mean / scale = standard deviation /!\
print("u_heat =", u_heat)
number_of_daytypes = len(set(elec.df.Day_type))
kappa = (np.random.dirichlet(size = 1, alpha = [1]*number_of_daytypes)[0])*number_of_daytypes
print("kappa =", kappa, ", sum kappa verif :", sum(kappa))
sigma2 = 1 / (np.random.gamma(size = 1, shape = 0.01, scale = 100)[0]) 
print('sigma2 = ', sigma2) #Très grand...
sigma2_s_param = 1 / (np.random.gamma(size = 1, shape = 0.01, scale = 100)[0])
sigma2_g_param = 1 / (np.random.gamma(size = 1, shape = 0.01, scale = 100)[0])

u_heat = 14.4714351637
kappa = [ 0.41312159  1.1040987   1.08632684  0.22836172  0.2321909   1.16154018
  2.27696106  1.49739901] , sum kappa verif : 8.0
sigma2 =  4.30397970916e+42


In [195]:
class Particle_Filtering_Model(object):
    
    def __init__(self,u_heat,kappa,sigma2, sigma2_s_param, sigma2_g_param, nb_particles, len_filtering):
        """constructor
        u_heat,kappa,sigma2, sigma2_s_param, sigma2_g_param : float which define the fixed parameters of the model"""
        import numpy as np
        from scipy.stats import truncnorm
        import math
        #Define the fixed parameters theta of the model
        self.u_heat = u_heat
        self.kappa = kappa
        self.sigma2 = sigma2
        self.sigma2_s_param = sigma2_s_param
        self.sigma2_g_param = sigma2_g_param
        #Define the number of particles we want to simulate and the length of the filtering process (from date 0 to date n0)
        self.nb_particles = nb_particles
        self.len_filtering = len_filtering
        #Initialize the numpy arrays of x, s, g_heat, sigma_s and sigma_g
        self.s = np.zeros((len_filtering, nb_particles))
        self.sigma_s = np.zeros((len_filtering, nb_particles))
        self.sigma_g = np.zeros((len_filtering, nb_particles))
        self.g_heat = np.zeros((len_filtering, nb_particles))
        self.x = np.zeros((len_filtering, nb_particles))
        #Initialize the weights
        self.lw = np.zeros((len_filtering, nb_particles))
        self.w = np.zeros((len_filtering, nb_particles))
    
    def exp_and_normalize(self, lw):
        """Computes the normalized weights from the non normalized log weights
        lw : vector of the log weights (row numpy array)"""
        w = np.exp(lw - max(lw)) #np.exp Calculates the exponential of all elements in the input array.
        res = w / sum(w)
        return(res)
    
    def compute_log_g_y(self, x, elec):
        """ Computes the log of the likelihood function 
        x : float
        elec : float """
        res = -((elec-x)**2)/(2*self.sigma2)
        return(res)
    
    def vcompute_log_g_y(self, x_vector, elec):
        """ Computes the log of the likelihood function for all particles at the same time
        x : row of numpy array (vector of the values of x for each particle at the considered time)
        elec : float (value of the observed electricty load y at the considered time) """
        vectorized_function = np.vectorize(self.compute_log_g_y, excluded=['elec'])
        return(vectorized_function(x_vector, elec))
    
    def compute_x(self, s, g_heat, temperature, daytype):
        """ Computes the value of x for each particle at a specific time t
        Parameters :
        -s are g_heat are row vectors (value of s and g_heat for each particle)
        -temperature and daytype are constants (observed data at time t)
        Returns a row vector"""
        x_season = self.kappa[daytype]*s
        if temperature < u_heat :
            x_heat = (temperature-self.u_heat)*g_heat
        else :
            x_heat = np.zeros(g_heat.shape)
        x = x_season + x_heat
        return(x)
    
    def initialization_SIS_withoutMCMC(self, temperature_ts, daytype_ts):
        """ Initialization of the values of s, g_heat, sigma_s and sigma_g (and hence x) using the prior distributions of the article
        This initialization could also be done using a first MCMC Gibbs sampling step"""
        self.s[0,] = truncnorm.rvs(a = 0,b = math.inf, loc= 0, scale = 10**4, size=self.nb_particles)#loc = mean / scale = standard deviation /!\
        self.sigma_s[0,] =(1 / (np.random.gamma(size = self.nb_particles, shape = 0.01, scale = 100)))**0.5
        self.sigma_g[0,] = (1 / (np.random.gamma(size = self.nb_particles, shape = 0.01, scale = 100)))**0.5
        self.g_heat[0,] = truncnorm.rvs(a = - math.inf,b = 0, loc= 0, scale = 10**4, size=self.nb_particles)#loc = mean / scale = standard deviation /!\
        self.x[0,] = self.compute_x(s = self.s[0,], g_heat = self.g_heat[0,], temperature = temperature_ts[0], daytype = daytype_ts[0])
        
    def sample_new_sigma_g(self, sigma_g_prev):
        """ Simulates the current sigma_g given the previous sigma_g
        Parameters :
        -sigma_g_prev : float 
        Return a float"""
        #see https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.truncnorm.html
        #/!\ The standard form of this distribution is a STANDARD normal truncated to the range [a, b] — 
        #notice that a and b are defined over the domain of the STANDARD normal. 
        #To convert clip values for a specific mean and standard deviation :
        my_mean = 0
        my_std = (self.sigma2_g_param)**0.5
        myclip_a = - sigma_g_prev
        myclip_b = math.inf
        a, b = (myclip_a - my_mean) / my_std, (myclip_b - my_mean) / my_std
        error = truncnorm.rvs(a = a, b = b, loc= my_mean, scale = my_std, size=1)[0]#loc = mean / scale = standard deviation
        res = sigma_g_prev + error
        return(res)
    
    def vsample_new_sigma_g(self, sigma_g_prev_vector):
        """ Simulates the current sigma_g for each particle given the previous sigma_g for each particle
        Parameters : 
        -sigma_g_prev : vector (row of a numpy array)
        Returns a row vector"""
        vectorized_function = np.vectorize(self.sample_new_sigma_g)
        return(vectorized_function(sigma_g_prev_vector))
    
    def sample_new_sigma_s(self, sigma_s_prev):
        """ Simulates the current sigma_s given the previous sigma_s
        Parameters : 
        -sigma_s_prev : float 
        Returns a float """
        my_mean = 0
        my_std = (self.sigma2_s_param)**0.5
        myclip_a = - sigma_s_prev
        myclip_b = math.inf
        a, b = (myclip_a - my_mean) / my_std, (myclip_b - my_mean) / my_std
        error = truncnorm.rvs(a = a, b = b, loc= my_mean, scale = my_std, size=1)[0]#loc = mean / scale = standard deviation
        res = sigma_s_prev + error
        return(res)
    
    def vsample_new_sigma_s(self, sigma_s_prev_vector) :
        """ Simulates the current sigma_s for each particle given the previous sigma_s for each particle
        Parameters : 
        -sigma_s_prev : vector (row of a numpy array) 
        Returns a row vector """
        vectorized_function = np.vectorize(self.sample_new_sigma_s)
        return(vectorized_function(sigma_s_prev_vector))
    
    def sample_new_g_heat(self, g_heat_prev, sigma_g_current):
        """ Simulates the current g_heat given the previous g_heat and the current sigma_g
        Parameters :
        -g_heat_prev : float
        -sigma_g_current : float
        Returns a float """
        my_mean = 0
        my_std = sigma_g_current
        myclip_a = - math.inf
        myclip_b = -g_heat_prev
        a, b = (myclip_a - my_mean) / my_std, (myclip_b - my_mean) / my_std
        error = truncnorm.rvs(a = a, b = b, loc= my_mean, scale = my_std, size=1)#loc = mean / scale = standard deviation
        res = g_heat_prev + error
        return(res)
    
    def vsample_new_g_heat(self, g_heat_prev, sigma_g_current):
        """ Simulates the current g_heat for each particle given the previous g_heat for each particle and the current 
        sigma_g for each particle.
        Parameters :
        -g_heat_prev :row of a numpy array (size =  1 x nb_particles)
        -sigma_g_current : row of a numpy array (size = 1 x nb_particles)
        Returns a row vector"""
        nb_part = len(g_heat_prev)
        g_heat_new =  np.zeros((1, nb_part))
        for j in range(nb_part):
            g_heat_new[0,j] = self.sample_new_g_heat(g_heat_prev[j], sigma_g_current[j])
        return(g_heat_new)
    
    def sample_new_s(self, s_prev, sigma_s_current):
        """ Simulates the current s given the previous s and the current sigma_s
        Parameters :
        -s_prev : float
        -sigma_s_current : float"""
        my_mean = 0
        my_std = sigma_s_current
        myclip_a = -s_prev
        myclip_b = math.inf
        a, b = (myclip_a - my_mean) / my_std, (myclip_b - my_mean) / my_std
        error = truncnorm.rvs(a = a, b = b, loc= my_mean, scale = my_std, size=1)#loc = mean / scale = standard deviation
        res = s_prev + error
        return(res)
    
    def vsample_new_s(self, s_prev, sigma_s_current):
        """ Simulates the current s for each particle given the previous s for each particle and the current 
        sigma_s for each particle.
        Parameters :
        -s_prev :row of a numpy array (size =  1 x nb_particles)
        -sigma_s_current : row of a numpy array (size = 1 x nb_particles)"""
        nb_part = len(s_prev)
        s_new =  np.zeros((1, nb_part))
        for j in range(nb_part):
            s_new[0,j] = self.sample_new_s(s_prev[j], sigma_s_current[j])
        return(s_new)
    
    def sample_from_transition(self, s_prev, g_heat_prev, sigma_s_prev, sigma_g_prev):
        """ Samples the vector x, s, g_heat, sigma2_s, sigma2_g at time n from transition density
        Given s, g_heat, sigma2_s, sigma2_g at time (n-1)
        Parameters :
        s_prev, g_heat_prev, sigma2_s_prev, sigma2_g_prev : row of numpy array (size = 1 x nb_particles)"""
        sigma_g_current = self.vsample_new_sigma_g(sigma_g_prev)
        sigma_s_current = self.vsample_new_sigma_s(sigma_s_prev)
        g_heat_current = self.vsample_new_g_heat(g_heat_prev, sigma_g_current)
        s_current = self.vsample_new_s(s_prev, sigma_s_current)
        return(s_current, g_heat_current, sigma_s_current, sigma_g_current)
    
    def SIS_filter(self, elec_ts, temperature_ts, daytype_ts):
        """ Sequential Importance Sampling for filtering
        Given the set of parameters, simulate particles for each day until today (past)"""
        #Initialization : time n = 0 
        self.initialization_SIS_withoutMCMC(temperature_ts, daytype_ts)
        self.lw[0,] = self.vcompute_log_g_y(x_vector = self.x[0,], elec = elec_ts[0]) #Compute the log weights at time 0
        self.w[0,] = self.exp_and_normalize(self.lw[0,])#Compute the normalized weights at time 0
        self.lw[0,] = np.log(self.w[0,]) #Compute the log of the normalized weights at time 0
        #Main : at time n > 0
        for n in range(1,self.len_filtering,1):
            #1. Sample from transition density
            self.s[n,], self.g_heat[n,], self.sigma_s[n,], self.sigma_g[n,] = self.sample_from_transition(s_prev = self.s[(n-1),], 
                                g_heat_prev = self.g_heat[(n-1),], sigma_s_prev = self.sigma_s[(n-1),], 
                                sigma_g_prev = self.sigma_g[(n-1),])
            self.x[n,] = self.compute_x(s = self.s[n,], g_heat = self.g_heat[n,],  temperature = temperature_ts[n], daytype = daytype_ts[n])
            #2. Update the weights
            self.lw[n,] = self.lw[(n-1),] + self.vcompute_log_g_y(x_vector = self.x[n,], elec = elec_ts[n]) #Compute the log weights at time n
            self.w[n,] = self.exp_and_normalize(self.lw[n,])#Compute the normalized weights at time 0
            self.lw[n,] = np.log(self.w[n,]) #Compute the log of the normalized weights at time 0
        

In [203]:
test = Particle_Filtering_Model(u_heat,kappa,sigma2, sigma2_s_param, sigma2_g_param, nb_particles = 5, len_filtering = 3)

In [204]:
test.SIS_filter(elec_ts = elec_ts, temperature_ts = tp_ts, daytype_ts = daytype_ts)

In [205]:
test.x

array([[  1.40370449e+04,   3.08698842e+04,   5.50723635e+04,
          2.46475107e+05,   2.53368893e+04],
       [  6.67317832e+62,   5.13160960e+20,   2.19205510e+48,
          3.25627363e+20,   6.09276705e+59],
       [  1.34568956e+64,   3.87106371e+21,   3.91161596e+48,
          1.85600285e+21,   2.50880778e+60]])

In [206]:
test.w

array([[ 0.2       ,  0.2       ,  0.2       ,  0.2       ,  0.2       ],
       [ 0.        ,  0.49543165,  0.        ,  0.50456835,  0.        ],
       [ 0.        ,  0.20441286,  0.        ,  0.79558714,  0.        ]])

In [207]:
sum(test.w[2,])

1.0

In [208]:
test.sigma_s

array([[  2.90186048e+02,   8.64619225e+06,   3.54235969e+01,
          2.02683302e+04,   3.70405320e+60],
       [  1.34408444e+07,   1.43516727e+07,   5.09307073e+05,
          1.56718662e+07,   3.70405320e+60],
       [  1.69119078e+07,   9.90047393e+06,   2.55194264e+07,
          3.08126163e+07,   3.70405320e+60]])

In [209]:
test.s

array([[  3.63323932e+03,   7.33433662e+03,   1.20874381e+04,
          1.55621734e+04,   9.10405775e+03],
       [  6.34489896e+06,   6.58874946e+05,   7.74609059e+05,
          1.53630450e+07,   4.06890014e+59],
       [  7.55947444e+06,   7.28931217e+06,   3.22495918e+06,
          4.15310155e+07,   6.07280730e+60]])

In [210]:
test.sigma_g

array([[  1.62873447e+63,   4.24083627e+18,   1.63872469e+47,
          7.68573371e-02,   4.93979841e+01],
       [  1.62873447e+63,   5.79525768e+20,   1.63872469e+47,
          6.56772884e+19,   4.10667667e+20],
       [  1.62873447e+63,   4.44115776e+20,   1.63872469e+47,
          2.84427112e+20,   2.99747451e+19]])

In [211]:
test.g_heat

array([[ -5.86374394e+02,  -1.44143388e+03,  -2.80250152e+03,
         -2.14681437e+04,  -4.68678778e+02],
       [ -6.61742278e+61,  -5.08873413e+19,  -2.17374010e+47,
         -3.22906691e+19,  -3.36209968e+20],
       [ -1.14218706e+63,  -3.28566037e+20,  -3.32008009e+47,
         -1.57532799e+20,  -3.15656773e+20]])

In [212]:
test.compute_log_g_y(x = 10, elec = 2)

-7.4349792894921644e-42

In [213]:
test.compute_log_g_y(x = 500, elec = 2)

-2.8811009432987727e-38

In [214]:
test.vcompute_log_g_y(x_vector = [10, 500], elec = 2)

array([ -7.43497929e-42,  -2.88110094e-38])